We want to start a cluster with MPI capability.  
Bottom line is, you need to have a MPI capable cluster first.  
Unfortunatly, my local machines do NOT support this.  
MS provides a way to bulid cluster machine through MS HPC pack.  
However, the head node needs to run Windows Server OS, which I don't have.  
So, it can be done. 

We start a cluster with 3 MPI engines  
ipcluster start -n 3 --profile=mpi3

In [7]:
import ipyparallel as ipp
import numpy as np
c = ipp.Client(profile='mpi3', cluster_id='mpi3')
print(c.ids)

[0, 1, 2]


Now, start more engines!   
mpiexec -n 2 ipengine --profile=mpi3

In [8]:
c = ipp.Client(profile='mpi3', cluster_id='mpi3')
print(c.ids)

[0, 1, 2, 3, 4]


We have now 5 engines!

In [9]:
dview = c[:]
dview.scatter('a',np.arange(28,dtype='float'))

<AsyncResult: scatter>

In [10]:
dview['a']

[array([0., 1., 2., 3., 4., 5.]),
 array([ 6.,  7.,  8.,  9., 10., 11.]),
 array([12., 13., 14., 15., 16., 17.]),
 array([18., 19., 20., 21., 22.]),
 array([23., 24., 25., 26., 27.])]

In [11]:
dview.run('psum.py')

<AsyncResult: execute>

In [12]:
%px totalsum = psum(a)

In [13]:
dview['totalsum']

[array(153.), array(153.), array(153.), array(225.), array(225.)]

Too bad, they are appaprently belonging to two seperated MPI_COMM_WORLD!  


In [14]:
dview.run('rank_mpi.py')

<AsyncResult: execute>

In [15]:
%px totalhost = hostname()

In [16]:
dview['totalhost']

['DESKTOP-10711',
 'DESKTOP-10711',
 'DESKTOP-10711',
 'DESKTOP-10711',
 'DESKTOP-10711']

In [17]:
%px totalrank = myrank()

In [18]:
dview['totalrank']

[0, 1, 2, 1, 0]